In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import xgboost as xgb
import pandas as pd
import sys
import random

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submit = pd.read_csv('gender_submission.csv')

In [3]:
selected_features = ['Ticket','Pclass', 'Name','Sex','Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked']
X_train = train[selected_features]
X_test = test[selected_features]
y_train = train['Survived']

# 補缺值

In [4]:
X_train['Embarked'].fillna('S', inplace=True)
X_train['Family_Size'] = X_train['Parch'] + X_train['SibSp']
X_train['Title'] = X_train['Name'].str.split(", ", expand=True)[1]
X_train['Title'] = X_train['Title'].str.split(".", expand=True)[0]

X_test['Fare'].fillna(X_test['Fare'].mean(), inplace=True)
X_test['Family_Size'] = X_test['Parch'] + X_test['SibSp']
X_test['Title'] = X_test['Name'].str.split(", ", expand=True)[1]
X_test['Title'] = X_test['Title'].str.split(".", expand=True)[0]

/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
Ticket         891 non-null object
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       891 non-null object
Family_Size    891 non-null int64
Title          891 non-null object
dtypes: float64(2), int64(4), object(6)
memory usage: 83.6+ KB


In [6]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
Ticket         418 non-null object
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Fare           418 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
Family_Size    418 non-null int64
Title          418 non-null object
dtypes: float64(2), int64(4), object(6)
memory usage: 39.3+ KB


# Ticket Number

In [5]:
X_train['Ticket'] = X_train['Ticket'].apply(lambda x : x.replace(".","").replace("/","").strip().split(' ')[0] if not x.isdigit() else 'X')
X_train['Ticket'].unique()
X_test['Ticket'] = X_test['Ticket'].apply(lambda x : x.replace(".","").replace("/","").strip().split(' ')[0] if not x.isdigit() else 'X')
X_test['Ticket'].unique()

array(['X', 'A4', 'WEP', 'SCPARIS', 'STONO2', 'PC', 'C', 'A5', 'SCAH',
       'CA', 'WC', 'SOTONOQ', 'STONO', 'SCA3', 'FCC', 'FC', 'PP',
       'STONOQ', 'SCA4', 'SOPP', 'SOC', 'SOTONO2', 'AQ4', 'A', 'SC',
       'SCParis', 'LP', 'AQ3'], dtype=object)

# Cabin & CabinNum 

In [6]:
X_train['Cabin'].astype(str)
X_test['Cabin'].astype(str)

for i in range(len(X_train['Title'])):
    if sys.getsizeof(X_train['Cabin'][i]) == 54:
        X_train['Cabin'][i] = X_train['Cabin'][i][2:5]
    
    if sys.getsizeof(X_train['Cabin'][i]) == 58:
        X_train['Cabin'][i] = X_train['Cabin'][i][0:3]
    else:
        if sys.getsizeof(X_train['Cabin'][i]) >= 56:
            X_train['Cabin'][i] = X_train['Cabin'][i][0:3]
            
for i in range(len(X_test['Title'])):
    if sys.getsizeof(X_test['Cabin'][i]) == 54:
        X_test['Cabin'][i] = X_test['Cabin'][i][2:5]
    if sys.getsizeof(X_test['Cabin'][i]) == 58:
        X_test['Cabin'][i] = X_test['Cabin'][i][0:3]
    else:
        if sys.getsizeof(X_test['Cabin'][i]) >= 56:
            X_test['Cabin'][i] = X_test['Cabin'][i][0:3]
            
X_train["CabinNum"] = X_train['Cabin'].apply(lambda x : str(x)[1:] if not pd.isnull(x) else random.randint(1,50))
X_train["Cabin"] = X_train['Cabin'].apply(lambda x : str(x)[0] if not pd.isnull(x) else 'C')
X_test["CabinNum"] = X_test['Cabin'].apply(lambda x : str(x)[1:] if not pd.isnull(x) else '50')
X_test["Cabin"] = X_test['Cabin'].apply(lambda x : str(x)[0] if not pd.isnull(x) else 'C')

for i in range(len(X_test['CabinNum'])):
    if sys.getsizeof(X_test['CabinNum'][i]) == 49:
        X_test['CabinNum'][i] = '10'
    #if X_test['Cabin'][i] == 'T':
     #   X_test['Cabin'][i] = 'F'
for j in range(len(X_train['CabinNum'])):
    if sys.getsizeof(X_train['CabinNum'][j]) == 49:
        X_train['CabinNum'][j] = '10'
    #if X_train['Cabin'][j] == 'T':
     #   X_train['Cabin'][j] = 'F'
        
X_train['CabinNum']=X_train['CabinNum'].astype(str).astype(int)
X_test['CabinNum']=X_test['CabinNum'].astype(str).astype(int)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying t

# One Hot Encoding

In [7]:
X_train['Sex'] = X_train['Sex'].astype('category').cat.codes
X_test['Sex'] = X_test['Sex'].astype('category').cat.codes
X_train['Cabin'] = X_train['Cabin'].astype('category').cat.codes
X_test['Cabin'] = X_test['Cabin'].astype('category').cat.codes
X_train['Embarked'] = X_train['Embarked'].astype('category').cat.codes
X_test['Embarked'] = X_test['Embarked'].astype('category').cat.codes
X_train['Ticket'] = X_train['Ticket'].astype('category').cat.codes
X_test['Ticket'] = X_test['Ticket'].astype('category').cat.codes

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(X_train[['Sex','Cabin','Embarked','Ticket']])
enc.fit(X_test[['Sex','Cabin','Embarked','Ticket']])

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

# Age using ML:
Pclass	Sex	Age	Fare	SibSp	Parch	Family_Size	Cabin	CabinNum	Embarked

In [8]:
dataAgeNull = X_train[X_train["Age"].isnull()]
dataAgeNotNull = X_train[X_train["Age"].notnull()]
remove_outlier = dataAgeNotNull[(np.abs(dataAgeNotNull["Fare"]-dataAgeNotNull["Fare"].mean())>(4*dataAgeNotNull["Fare"].std()))|
                      (np.abs(dataAgeNotNull["Family_Size"]-dataAgeNotNull["Family_Size"].mean())>(4*dataAgeNotNull["Family_Size"].std()))                     
                     ]
rfModel_age = RandomForestRegressor(n_estimators=2000,random_state=42)
ageColumns = ['Embarked', 'Fare', 'Pclass', 'Sex', 'Family_Size','Cabin','Ticket']
rfModel_age.fit(remove_outlier[ageColumns], remove_outlier["Age"])

ageNullValues = rfModel_age.predict(X= dataAgeNull[ageColumns])
dataAgeNull.loc[:,"Age"] = ageNullValues
X_train = dataAgeNull.append(dataAgeNotNull)
X_train.reset_index(inplace=True, drop=True)

/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


# Age using average

In [8]:
X_train['Age'] = X_train['Age'].apply(lambda x : x if not pd.isnull(x) else 0)
for i in range(len(X_train['Title'])):
    
    if X_train['Age'][i] == 0:
        if X_train['Title'][i] == 'Capt':
            X_train['Age'][i] = 70.0
        if X_train['Title'][i] == 'Col':
            X_train['Age'][i] = 54.0
        if X_train['Title'][i] == 'Don':
            X_train['Age'][i] = 40.0
        if X_train['Title'][i] == 'Dona':
            X_train['Age'][i] = 39.0
        if X_train['Title'][i] == 'Dr':
            X_train['Age'][i] = 43.5
        if X_train['Title'][i] == 'Jonkheer':
            X_train['Age'][i] = 38.0
        if X_train['Title'][i] == 'Lady':
            X_train['Age'][i] = 48.0
        if X_train['Title'][i] == 'Major':
            X_train['Age'][i] = 48.5
        if X_train['Title'][i] == 'Master':
            X_train['Age'][i] = 5.48
        if X_train['Title'][i] == 'Miss':
            X_train['Age'][i] = 21.7
        if X_train['Title'][i] == 'Mlle':
            X_train['Age'][i] = 24.0
        if X_train['Title'][i] == 'Mme':
            X_train['Age'][i] = 24.0
        if X_train['Title'][i] == 'Mr':
            X_train['Age'][i] = 32.2
        if X_train['Title'][i] == 'Mrs':
            X_train['Age'][i] = 36.9
        if X_train['Title'][i] == 'Ms':
            X_train['Age'][i] = 28.0
        if X_train['Title'][i] == 'Rev':
            X_train['Age'][i] = 41.2
        if X_train['Title'][i] == 'Sir':
            X_train['Age'][i]= 49.0
        if X_train['Title'][i] == 'the Countess':
            X_train['Age'][i] = 33.0

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

In [9]:
X_test['Age'] = X_test['Age'].apply(lambda x : x if not pd.isnull(x) else 0)
for i in range(len(X_test['Title'])):
    
    if X_test['Age'][i] == 0:
        if X_test['Title'][i] == 'Capt':
            X_test['Age'][i] = 70.0
        if X_test['Title'][i] == 'Col':
            X_test['Age'][i] = 54.0
        if X_test['Title'][i] == 'Don':
            X_test['Age'][i] = 40.0
        if X_test['Title'][i] == 'Dona':
            X_test['Age'][i] = 39.0
        if X_test['Title'][i] == 'Dr':
            X_test['Age'][i] = 43.5
        if X_test['Title'][i] == 'Jonkheer':
            X_test['Age'][i] = 38.0
        if X_test['Title'][i] == 'Lady':
            X_test['Age'][i] = 48.0
        if X_test['Title'][i] == 'Major':
            X_test['Age'][i] = 48.5
        if X_test['Title'][i] == 'Master':
            X_test['Age'][i] = 5.48
        if X_test['Title'][i] == 'Miss':
            X_test['Age'][i] = 21.7
        if X_test['Title'][i] == 'Mlle':
            X_test['Age'][i] = 24.0
        if X_test['Title'][i] == 'Mme':
            X_test['Age'][i] = 24.0
        if X_test['Title'][i] == 'Mr':
            X_test['Age'][i] = 32.2
        if X_test['Title'][i] == 'Mrs':
            X_test['Age'][i] = 36.9
        if X_test['Title'][i] == 'Ms':
            X_test['Age'][i] = 28.0
        if X_test['Title'][i] == 'Rev':
            X_test['Age'][i] = 41.2
        if X_test['Title'][i] == 'Sir':
            X_test['Age'][i]= 49.0
        if X_test['Title'][i] == 'the Countess':
            X_test['Age'][i] = 33.0

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

In [13]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
Cabin          891 non-null object
Embarked       891 non-null object
Family_Size    891 non-null int64
Title          891 non-null object
CabinNum       891 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [10]:
X_train = X_train[['Pclass','Sex', 'Fare', 'SibSp', 'Parch']]
X_test = X_test[['Pclass','Sex', 'Fare', 'SibSp', 'Parch']]

In [ ]:
'SibSp', 'Parch','Ticket','Ticket','Age',,'Cabin','CabinNum', 

In [11]:
X_train

,Pclass,Sex,Fare,SibSp,Parch
0,3,1,7.2500,1,0
1,1,0,71.2833,1,0
2,3,0,7.9250,0,0
3,1,0,53.1000,1,0
4,3,1,8.0500,0,0
5,3,1,8.4583,0,0
6,1,1,51.8625,0,0
7,3,1,21.0750,3,1
8,3,0,11.1333,0,2
9,2,0,30.0708,1,0


In [17]:
X_test

,Pclass,Sex,Fare,SibSp,Parch
0,3,1,7.8292,0,0
1,3,0,7.0000,1,0
2,2,1,9.6875,0,0
3,3,1,8.6625,0,0
4,3,0,12.2875,1,1
5,3,1,9.2250,0,0
6,3,0,7.6292,0,0
7,2,1,29.0000,1,1
8,3,0,7.2292,0,0
9,3,1,24.1500,2,0


# Random Forest

In [15]:
rfc = RandomForestClassifier()

In [16]:
cross_val_score(rfc, X_train, y_train, cv=10).mean()

0.8115821132675066

In [67]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [68]:
survived_predict = rfc.predict(X_test)

In [69]:
submission2=pd.DataFrame({'PassengerId':test['PassengerId'], 'Survived':survived_predict})
submission2.to_csv('submission.csv', index=False)

# Xgboost 0.83843

In [18]:
model = XGBClassifier(max_depth=7, n_estimators=300) #tuned 

In [19]:
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [20]:
cross_val_score(model, X_train, y_train, cv=10).mean()

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

0.8059754851889684

In [21]:
y_pred = model.predict(X_test)

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [22]:
submission2=pd.DataFrame({'PassengerId':test['PassengerId'], 'Survived':y_pred})
submission2.to_csv('submission.csv', index=False)